First let's import some data.

In [2]:
import chess.pgn
import chess
import pandas as pd
import numpy as np
from pathlib import Path

from pandas import DataFrame

root = Path.cwd().parent
file_path = root / 'data/raw/lichess_db_standard_rated_2013-01.pgn'

pgn = open(file_path, encoding='utf-8')

game = chess.pgn.read_game(pgn)

Then let's try to see the board and the legal moves

In [3]:
board = game.board()

# Skip to move 10
for i, move in enumerate(game.mainline_moves(), start=1):
    board.push(move)
    if i == 10:
        break

print(board, '\n')
print(bin(board.pieces(chess.PAWN, chess.WHITE)))

r n . q k b . r
p b p p . p . p
. p . . p . . p
. . . . . . . .
. . . P P . . .
P . N . . . . .
. P P . . P P P
R . . Q K B N R 

0b11000000000011110011000000000


For model to learn a dataframe of states and moves made from these states is needed. Since we need this dataframe for model training later, the boards should be converted into 12x8x8 tensors.

In [4]:
def board_to_bitboard(board: chess.Board):
    pieces = [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]
    colors = [chess.WHITE, chess.BLACK]

    bitmask = np.zeros((12, 8, 8), dtype=np.uint8)

    for i, color in enumerate(colors):
        for j, piece in enumerate(pieces):
            piece_int = board.pieces(piece, color)
            for k in range(64):
                if piece_int & 1:
                    rank = k // 8
                    bitmask[i * 6 + j][rank][k - rank * 8] = 1
                piece_int >>= 1
    return bitmask